# MWS Secondary Target Selection

## MWS_SPECIAL_WDBINARY_SV

Maintainers:
  - Boris Gaensicke (selection/catalog)
  - Andrew Cooper (secondary target code)

In [6]:
ls ../raw

WD_BinaryCandidates_GALEX_Gaia.csv  WD_BinaryCandidates_Variability.csv


In [7]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/code'

Two target lists of WD binary candidates, one based on a GALEX + Gaia selection, and one which is a heterogeneous collection of variability-selected sources, which can not be expressed in any formulaic way (but is small, ~5000 all-sky).

The selection criteria are:

???

In [8]:
pwd

'/global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/code'

In [50]:
import os
import numpy as np
from astropy.table import Table, Column
import astropy.coordinates as coord
import astropy.units as u
import astropy.io.fits as fits

In [15]:
target_class    = 'MWS_SPECIAL_WDBINARY_SV'
input_filenames = 'WD_BinaryCandidates_GALEX_Gaia.csv', 'WD_BinaryCandidates_Variability.csv'

REF_CAT   = 'G'
REF_EPOCH = '2015.5'

In [16]:
# Relative paths
input_paths  = [os.path.join('..','raw',_) for _ in input_filenames]

# Separate bright and faint output
output_path = os.path.join('..','indata','{}.fits'.format(target_class))

output_dir  = os.path.dirname(output_path)
if not os.path.exists(output_dir):
    print('Making output directory: {}'.format(os.path.abspath(output_dir)))
    os.makedirs(dir)

In [63]:
coord.Angle(_d['ra'][0],unit='hourangle')

<Angle 0.00684167 hourangle>

In [70]:
_d['dec'][0]

'+33:25:42.98'

In [69]:
_d = d_list[1]
cc = coord.SkyCoord(_d['ra'],_d['dec'],unit=('hourangle','deg'))

dms_tuple(d=33.0, m=25.0, s=42.98000000000286)

In [73]:

d_list = list()
for _ in input_paths:
    print('Reading input: {}'.format(os.path.abspath(_)))
    _d = Table.read(_)
    print('Read {:d} rows'.format(len(_d)))
    if _d['ra'].dtype.name.startswith('str'):
        new_c = coord.SkyCoord(_d['ra'],_d['dec'],unit=('hourangle','deg'))
        _d['ra']  = new_c.ra.deg
        _d['dec'] = new_c.dec.deg
    d_list.append(_d)

Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/raw/WD_BinaryCandidates_GALEX_Gaia.csv
Read 128956 rows
Reading input: /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/raw/WD_BinaryCandidates_Variability.csv
Read 4695 rows


In [74]:
for _d in d_list:
    print('First 3 rows:')
    print(_d[0:3])
    print(_d.colnames)
    print(_d.dtype)
    print()

First 3 rows:
     source_id              ra         ...        pmdec        Epoch 
------------------- ------------------ ... ------------------- ------
4026850114257127040 181.78587423127559 ...   1.710741096587701 2015.5
5573594388757919616  95.04847173837307 ...  1.8740119901921664 2015.5
5672222574635034880 148.88690785784294 ... -18.823875423455547 2015.5
['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'Epoch']
[('source_id', '<i8'), ('ra', '<f8'), ('dec', '<f8'), ('pmra', '<f8'), ('pmdec', '<f8'), ('Epoch', '<f8')]

First 3 rows:
     source_id               ra         ...        pmdec        epoch 
------------------- ------------------- ... ------------------- ------
2875039670446086016 0.10262499999999998 ...  -5.206250210324438 2015.5
4684042885786567936               0.204 ... -3.7917165151605716 2015.5
 396306356208737792  0.2300833333333333 ... -1.7761193764687782 2015.5
['source_id', 'ra', 'dec', 'pmra', 'pmdec', 'epoch']
[('source_id', '<i8'), ('ra', '<f8'), ('dec', '<f8'),

In [90]:
import astropy.table as table
d_stack = table.vstack(d_list)
d_stack.columns

<TableColumns names=('source_id','ra','dec','pmra','pmdec','Epoch','epoch')>

In [93]:
uniq, uidx = np.unique(d_stack['source_id'],return_index=True)
nuniq      = len(uniq)
print('{} of {} targets are unique'.format(nuniq,len(d_stack)))
print('Removing {} duplicates'.format(len(d_stack)-nuniq))

d = d[uidx]

132995 of 133651 targets are unique
Removing 656 duplicates


No mag information for WDBINARY

In [94]:
bright_time_targets = np.arange(0,len(d))

Adjust to secondary target data model:

In [95]:
output = d['ra','dec','source_id'][bright_time_targets]
output.rename_columns(['ra','dec','source_id'],['RA','DEC','REF_ID'])
output.add_column(Column(np.repeat(False,len(output)),'OVERRIDE'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMRA'))
output.add_column(Column(np.zeros(len(output),dtype=np.float32),'PMDEC'))
output.add_column(Column(np.repeat(REF_CAT,len(output)),'REF_CAT'))
output.add_column(Column(np.repeat(REF_EPOCH,len(output)),'REF_EPOCH'))

Add header:

In [96]:
output.meta['VRAWDATA'] = int(os.path.abspath(input_paths[0]).split(os.path.sep)[-3])

In [97]:
output[0:3]

RA,DEC,REF_ID,OVERRIDE,PMRA,PMDEC,REF_CAT,REF_EPOCH
float64,float64,int64,bool,float32,float32,str1,str6
45.17405739851883,0.5437712864337338,54528905041920,False,0.0,0.0,G,2015.5
45.92584768471102,0.8861145695998326,152935195517952,False,0.0,0.0,G,2015.5
44.28743100663697,0.7743488847665643,288175125714560,False,0.0,0.0,G,2015.5


Write output:

In [98]:
print('Writing {:d} rows of output to {}'.format(len(output),os.path.abspath(output_path)))
output.write(output_path,overwrite=True)

Writing 132995 rows of output to /global/projecta/projectdirs/desi/external/mws/sv/MWS_SPECIAL_WDBINARY_SV/1/indata/MWS_SPECIAL_WDBINARY_SV.fits
